# This notebook demonstrates how ListPermissions works with LFTags. 
This notebook is example code, not meant for any other purpose other than demonstration. 

In [3]:
import boto3

REGION='us-east-1'

p_session = boto3.session.Session(profile_name='default')
p_glue = p_session.client('glue', region_name=REGION)
p_lf = p_session.client('lakeformation', region_name=REGION)

### Constants

In [4]:
DB_S3_LOCATION='s3://abcdefghi-us-east-1/lfpermslisttest/'
DATABASE_NAME='lf_list_perms_test'
TABLE_NAME='lf_tag_test_table1'
LF_TAG_1_NAME='list_perm_tag_1'
LF_TAG_2_NAME='list_perm_tag_2'
USER_ARN='arn:aws:iam::123456789012:role/admin'

### Create my database and table if they dont exist

In [5]:
try:
    p_glue.create_database(
        DatabaseInput = {
            'Name': DATABASE_NAME,
            'LocationUri' : DB_S3_LOCATION
        }
    )
except Exception as e:
    print(e)

try:
    p_glue.create_table(
        DatabaseName=DATABASE_NAME,
        TableInput={
            'Name': TABLE_NAME,
            'StorageDescriptor': {
                'Columns': [
                    {'Name': 'customer_id','Type': 'bigint'},
                    {'Name': 'prefix','Type': 'string'},
                    {'Name': 'first_name','Type': 'string'},
                    {'Name': 'middle_name','Type': 'string'},
                    {'Name': 'last_name','Type': 'string'},
                    {'Name': 'suffix','Type': 'string'},
                    {'Name': 'gender','Type': 'string'},
                    {'Name': 'dob','Type': 'string'},
                    {'Name': 'phone','Type': 'string'},
                    {'Name': 'building_number','Type': 'bigint'},
                    {'Name': 'street_name','Type': 'string'},
                    {'Name': 'city','Type': 'string'},
                    {'Name': 'cust_zip','Type': 'bigint'},
                    {'Name': 'country','Type': 'string' }
                ],
                'Location': '',
                'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
                'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
                'Compressed': False,
                'NumberOfBuckets': 0,
                'SerdeInfo': {
                    'SerializationLibrary': 'org.apache.hadoop.hive.serde2.OpenCSVSerde',
                    'Parameters': {
                        'separatorChar': ','
                    }
                },
                'SortColumns': [],
                'StoredAsSubDirectories': False
            },
            'PartitionKeys': [],
            'TableType': 'EXTERNAL_TABLE',
            'Parameters': {
                'classification': 'csv',
                'lakeformation.aso.status': 'false'
            }
        }
    )
except Exception as e:
    print(e)


An error occurred (AlreadyExistsException) when calling the CreateDatabase operation: Database already exists.
An error occurred (AlreadyExistsException) when calling the CreateTable operation: Table already exists.


### Create my tags if they dont exist.

In [6]:
try:
    p_lf.create_lf_tag(
        TagKey=LF_TAG_1_NAME,
        TagValues=[
            'val1',
            'val2',
            'val3'
        ]
    )
except Exception as e:
    print(e)

try:
    p_lf.create_lf_tag(
        TagKey=LF_TAG_2_NAME,
        TagValues=[
            'val1',
            'val2',
            'val3'
        ]
    )
except Exception as e:
    print(e)


An error occurred (InvalidInputException) when calling the CreateLFTag operation: Tag key already exists
An error occurred (InvalidInputException) when calling the CreateLFTag operation: Tag key already exists


### Associate the tags to my resources.

In [7]:
p_lf.add_lf_tags_to_resource(
    Resource={
        'Database': {
            'Name': DATABASE_NAME
        }
    },
    LFTags=[
        {
            'TagKey':  LF_TAG_1_NAME,
            'TagValues': [
                'val1'
            ]
        },
        {
            'TagKey':  LF_TAG_2_NAME,
            'TagValues': [
                'val2'
            ]
        }
    ]
)

p_lf.add_lf_tags_to_resource(
    Resource={
        'Table': {
            'DatabaseName' : DATABASE_NAME,
            'Name': TABLE_NAME
        }
    },
    LFTags=[
        {
            'TagKey':  LF_TAG_1_NAME,
            'TagValues': [
                'val2'
            ]
        },
        {
            'TagKey':  LF_TAG_2_NAME,
            'TagValues': [
                'val1'
            ]
        }
    ]
)

{'ResponseMetadata': {'RequestId': '8e17a84b-afeb-4c4d-8513-d2d480c71182',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 05 Oct 2023 20:14:44 GMT',
   'content-type': 'application/json',
   'content-length': '15',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8e17a84b-afeb-4c4d-8513-d2d480c71182',
   'cache-control': 'no-cache'},
  'RetryAttempts': 0},
 'Failures': []}

### Grant permissions to my user
User will have two grants with the following expressions: 
* Database permission - LFTag Expression "LF_TAG1 = val1 AND LFTAG2 = val2"
* Table permissions - LFTag Expression "LF_TAG1 = val1 AND LFTAG2 = val2"
* Table permissions - LFTag Expression "LF_TAG1 = (val2 OR val3) AND LFTAG2 = (val3)"

In [12]:
p_lf.grant_permissions(
    Principal={
        'DataLakePrincipalIdentifier': USER_ARN
    },
    Resource={
        'LFTagPolicy': {
            'ResourceType': 'DATABASE',
            'Expression' : [
                {
                    'TagKey': LF_TAG_1_NAME,
                    'TagValues': [
                        'val1'
                    ]
                },
                {
                    'TagKey': LF_TAG_2_NAME,
                    'TagValues': [
                        'val2'
                    ]
                }
            ]
        }
    },
    Permissions=['DESCRIBE', 'ALTER']
)


p_lf.grant_permissions(
    Principal={
        'DataLakePrincipalIdentifier': USER_ARN
    },
    Resource={
        'LFTagPolicy': {
            'ResourceType': 'TABLE',
            'Expression': [
                {
                    'TagKey': LF_TAG_1_NAME,
                    'TagValues': [
                        'val1'
                    ]
                },
                {
                    'TagKey': LF_TAG_2_NAME,
                    'TagValues': [
                        'val2'
                    ]
                }
            ]
        }
    },
    Permissions=['SELECT', 'DESCRIBE', 'ALTER']
)

p_lf.grant_permissions(
    Principal={
        'DataLakePrincipalIdentifier': USER_ARN
    },
    Resource={
        'LFTagPolicy': {
            'ResourceType': 'TABLE',
            'Expression': [
                {
                    'TagKey': LF_TAG_1_NAME,
                    'TagValues': [
                        'val2',
                        'val3'
                    ]
                },
                {
                    'TagKey': LF_TAG_2_NAME,
                    'TagValues': [
                        'val3'
                    ]
                }
            ]
        }
    },
    Permissions=['SELECT', 'DESCRIBE', 'ALTER']
)

{'ResponseMetadata': {'RequestId': '26a8fdb6-d5e5-493e-831f-6cae7f4fc9aa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 05 Oct 2023 20:17:10 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '26a8fdb6-d5e5-493e-831f-6cae7f4fc9aa',
   'cache-control': 'no-cache'},
  'RetryAttempts': 0}}

### ListPermissions Behavior 
When using the ListPermissions API, and using filters, there must be an exact match for the filtering to occur. Partial searches are not supported yet. Below will demonstrate the behavior with different filters.

This block shows the current LF Tag Policies that exists in my account. The three that I created above.

In [13]:
perms = p_lf.list_permissions(
    ResourceType='LF_TAG_POLICY'
)

for perm in perms['PrincipalResourcePermissions']:
    print(perm)

{'Principal': {'DataLakePrincipalIdentifier': 'arn:aws:iam::123456789012:role/admin'}, 'Resource': {'LFTagPolicy': {'CatalogId': '123456789012', 'ResourceType': 'DATABASE', 'Expression': [{'TagKey': 'list_perm_tag_2', 'TagValues': ['val2']}, {'TagKey': 'list_perm_tag_1', 'TagValues': ['val1']}]}}, 'Permissions': ['ALTER', 'DESCRIBE'], 'PermissionsWithGrantOption': [], 'LastUpdated': datetime.datetime(2023, 10, 5, 16, 17, 10, 67000, tzinfo=tzlocal()), 'LastUpdatedBy': 'arn:aws:iam::123456789012:user/abcdefghi'}
{'Principal': {'DataLakePrincipalIdentifier': 'arn:aws:iam::123456789012:role/admin'}, 'Resource': {'LFTagPolicy': {'CatalogId': '123456789012', 'ResourceType': 'TABLE', 'Expression': [{'TagKey': 'list_perm_tag_2', 'TagValues': ['val2']}, {'TagKey': 'list_perm_tag_1', 'TagValues': ['val1']}]}}, 'Permissions': ['ALTER', 'DESCRIBE', 'SELECT'], 'PermissionsWithGrantOption': [], 'LastUpdated': datetime.datetime(2023, 10, 5, 16, 17, 10, 334000, tzinfo=tzlocal()), 'LastUpdatedBy': 'arn

### ListPermissions with DataLakePrincipalIdentifier
When DataLakePrincipalIdentifer (DLPI) is used as a filter, the behavior of the API changes. Here are some important differences:
* If DLPI is specified, you must specify the Resource field. For LF tags, it would be a database, table, LF-Tag Policy, LF Tag permissions, or any other permission types. 
* The ListPermissions API may contain some additional policies that ListPermissions without the DLPI may now show. This is because when DLPI is specified, the service provides some (not all) implicit permissions. For example, my user is a LF-Admin and calling ListPermissions on a table or database will provide Grantable permissions that was not explicity granted. This is because LF-Admins (non read-only) have grantable permissions on everything in the GDC.

In [17]:
perms = p_lf.list_permissions(
    Principal={
        'DataLakePrincipalIdentifier': USER_ARN
    },
    ResourceType='LF_TAG_POLICY',
    Resource={
        'LFTagPolicy': {
            'ResourceType': 'TABLE',
            'Expression': [
                {
                    'TagKey': LF_TAG_1_NAME,
                    'TagValues': [
                        'val2',
                        'val3'
                    ]
                },
                {
                    'TagKey': LF_TAG_2_NAME,
                    'TagValues': [
                        'val3'
                    ]
                }
            ]
        }
    }
)

for perm in perms['PrincipalResourcePermissions']:
    print(perm)

# As you can see, the grantable permissions are listed even though we have not explicity granted it because USER_ARN is a LF admin.

{'Principal': {'DataLakePrincipalIdentifier': 'arn:aws:iam::123456789012:role/admin'}, 'Resource': {'LFTagPolicy': {'CatalogId': '123456789012', 'ResourceType': 'TABLE', 'Expression': [{'TagKey': 'list_perm_tag_1', 'TagValues': ['val3', 'val2']}, {'TagKey': 'list_perm_tag_2', 'TagValues': ['val3']}]}}, 'Permissions': ['ALTER', 'DESCRIBE', 'SELECT'], 'PermissionsWithGrantOption': ['ALL', 'ALTER', 'DELETE', 'DESCRIBE', 'DROP', 'INSERT', 'SELECT']}


### Exact Expression Match
In this block, we are listing permissinos based on the tag expression (LF_TAG1 = (val2 OR val3) AND LFTAG2 = (val3)). This returns 1 policy as expected because the expression exactly matches the expression in the policy that we created.

In [11]:
perms = p_lf.list_permissions(
    ResourceType='LF_TAG_POLICY',
    Resource={
        'LFTagPolicy': {
            'ResourceType': 'TABLE',
            'Expression': [
                {
                    'TagKey': LF_TAG_1_NAME,
                    'TagValues': [
                        'val2',
                        'val3'
                    ]
                },
                {
                    'TagKey': LF_TAG_2_NAME,
                    'TagValues': [
                        'val3'
                    ]
                }
            ]
        }
    }
)

for perm in perms['PrincipalResourcePermissions']:
    print(perm)

{'Principal': {'DataLakePrincipalIdentifier': 'arn:aws:iam::123456789012:role/admin'}, 'Resource': {'LFTagPolicy': {'CatalogId': '123456789012', 'ResourceType': 'TABLE', 'Expression': [{'TagKey': 'list_perm_tag_2', 'TagValues': ['val3']}, {'TagKey': 'list_perm_tag_1', 'TagValues': ['val3']}]}}, 'Permissions': ['ALTER', 'DESCRIBE', 'SELECT'], 'PermissionsWithGrantOption': [], 'LastUpdated': datetime.datetime(2023, 10, 4, 19, 13, 58, 194000, tzinfo=tzlocal()), 'LastUpdatedBy': 'arn:aws:iam::123456789012:user/abcdefghi'}


### Partial search for an LF-Tag (Single Tag)
The following shows that we are searching only for a single LFTag and a single value (LF Tag Expression: "LFTAG_1 = val3"). This will not return any results because it does not exactly match the expression in my policy. 

In [10]:
perms = p_lf.list_permissions(
    ResourceType='LF_TAG_POLICY',
    Resource={
        'LFTagPolicy': {
            'ResourceType': 'TABLE',
            'Expression': [
                {
                    'TagKey': LF_TAG_1_NAME,
                    'TagValues': [
                        'val3'
                    ]
                }
            ]
        }
    }
)

for perm in perms['PrincipalResourcePermissions']:
    print(perm)

#No results is expected.

### Partial search for an LF-Tag (Multiple Tags)
The following shows that we are searching only for two LFTags and a single value (LF Tag Expression: "LFTAG_1 = val3 and LFTAG_2 = val3"). This will not return any results because it does not exactly match the expression in my policy. 

In [14]:
perms = p_lf.list_permissions(
    ResourceType='LF_TAG_POLICY',
    Resource={
        'LFTagPolicy': {
            'ResourceType': 'TABLE',
            'Expression': [
                {
                    'TagKey': LF_TAG_1_NAME,
                    'TagValues': [
                        'val3'
                    ]
                },            
                {
                    'TagKey': LF_TAG_2_NAME,
                    'TagValues': [
                        'val3'
                    ]
                }
            ]
        }
    }
)

for perm in perms['PrincipalResourcePermissions']:
    print(perm)

### Doing a Partial Search
If your use case is to find all the LF-Tag expressions where LFTAG_1 contains the value "val1", then you will have to iterate through all permissions, and filter the results based on what you are looking for. The LF service does not currently have any APIs that provides this functionality. 

In [15]:
perms = p_lf.list_permissions(
    ResourceType='LF_TAG_POLICY'
)

for perm in perms['PrincipalResourcePermissions']:
    for expression in perm['Resource']['LFTagPolicy']['Expression']:
        if expression['TagKey'] == LF_TAG_1_NAME:
            for tag_val in expression['TagValues']:
                if tag_val == 'val1':
                    print(f"Found policy: {perm}")

Found policy: {'Principal': {'DataLakePrincipalIdentifier': 'arn:aws:iam::123456789012:role/admin'}, 'Resource': {'LFTagPolicy': {'CatalogId': '123456789012', 'ResourceType': 'DATABASE', 'Expression': [{'TagKey': 'list_perm_tag_2', 'TagValues': ['val2']}, {'TagKey': 'list_perm_tag_1', 'TagValues': ['val1']}]}}, 'Permissions': ['ALTER', 'DESCRIBE'], 'PermissionsWithGrantOption': [], 'LastUpdated': datetime.datetime(2023, 10, 5, 16, 17, 10, 67000, tzinfo=tzlocal()), 'LastUpdatedBy': 'arn:aws:iam::123456789012:user/abcdefghi'}
Found policy: {'Principal': {'DataLakePrincipalIdentifier': 'arn:aws:iam::123456789012:role/admin'}, 'Resource': {'LFTagPolicy': {'CatalogId': '123456789012', 'ResourceType': 'TABLE', 'Expression': [{'TagKey': 'list_perm_tag_2', 'TagValues': ['val2']}, {'TagKey': 'list_perm_tag_1', 'TagValues': ['val1']}]}}, 'Permissions': ['ALTER', 'DESCRIBE', 'SELECT'], 'PermissionsWithGrantOption': [], 'LastUpdated': datetime.datetime(2023, 10, 5, 16, 17, 10, 334000, tzinfo=tzloc

## Cross Account Scenarios
In these scenarios, I have another account (Account2) that has their own LF tags. Those LF-Tags are used to share resources to my account (Account1), and also shares two LF-Tags with ASSOCIATE and DESCRIBE permissions (). 
In my account, I create a resource link to a shared resource, and associate my accounts LF-Tags to the resource link. I then grant permissions on the expression to users using the same LFexpression that the other account used to share with my account. 

In [21]:
# Constants
ACC2_ACCOUNT_ID='987654321098'
ACC2_LF_TAG1_NAME='list_perm_tag_1'
ACC2_LF_TAG2_NAME='list_perm_tag_2'
ACC2_SHARED_DATABASE_NAME='lftags_test_database'
DB_RL_NAME=ACC2_SHARED_DATABASE_NAME

In [19]:
tags = p_lf.list_lf_tags(
    ResourceShareType='FOREIGN'
)

for tag in tags['LFTags']:
    print(tag)

{'CatalogId': '987654321098', 'TagKey': 'list_perm_tag_1', 'TagValues': ['val2', 'val1']}
{'CatalogId': '987654321098', 'TagKey': 'list_perm_tag_2', 'TagValues': ['val2']}


### Create Resource link to shared database.

In [23]:
p_glue.create_database(
    DatabaseInput={
        'Name': DB_RL_NAME,
        'TargetDatabase': {
            'CatalogId': ACC2_ACCOUNT_ID,
            'DatabaseName': ACC2_SHARED_DATABASE_NAME
        }
    }
)

{'ResponseMetadata': {'RequestId': 'b851d86e-5048-4d82-9bd6-b3ec91edef3e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 05 Oct 2023 23:22:40 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b851d86e-5048-4d82-9bd6-b3ec91edef3e'},
  'RetryAttempts': 0}}

### Look at the tags associated with the shared database from Account 2.

In [28]:
lf_tags = p_lf.get_resource_lf_tags(
    Resource={
        'Database': {
            'CatalogId' : ACC2_ACCOUNT_ID,
            'Name': ACC2_SHARED_DATABASE_NAME
        }
    },
    ShowAssignedLFTags=True
)

for tag in lf_tags['LFTagOnDatabase']:
    print(tag)

{'CatalogId': '987654321098', 'TagKey': 'list_perm_tag_1', 'TagValues': ['val2']}
{'CatalogId': '987654321098', 'TagKey': 'list_perm_tag_2', 'TagValues': ['val2']}


### Associate Tags LFTAG_1:val1 and LFTAG_2:val2 to Resource link
User was already granted this expression so they should see the resource link.

In [29]:
p_lf.add_lf_tags_to_resource(
    Resource={
       'Database': {
            'Name': DB_RL_NAME
        }
    },
    LFTags=[
        {
            'TagKey': LF_TAG_1_NAME,
            'TagValues': [
                'val1'
            ]
        },
        {
            'TagKey': LF_TAG_2_NAME,
            'TagValues': [
                'val2'
            ]
        }
    ] 
)

{'ResponseMetadata': {'RequestId': '700d25ea-fae5-4913-8edf-069994292c79',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 06 Oct 2023 00:24:47 GMT',
   'content-type': 'application/json',
   'content-length': '15',
   'connection': 'keep-alive',
   'x-amzn-requestid': '700d25ea-fae5-4913-8edf-069994292c79',
   'cache-control': 'no-cache'},
  'RetryAttempts': 0},
 'Failures': []}

### List Permissions provided by the cross account (Account2)

In [36]:
perms = p_lf.list_permissions(
    ResourceType='LF_TAG_POLICY'
)

for perm in perms['PrincipalResourcePermissions']:
    if perm['Resource']['LFTagPolicy']['CatalogId'] == ACC2_ACCOUNT_ID:
        print(perm)

{'Principal': {'DataLakePrincipalIdentifier': 'arn:aws:iam::123456789012:role/admin'}, 'Resource': {'LFTagPolicy': {'CatalogId': '987654321098', 'ResourceType': 'DATABASE', 'Expression': [{'TagKey': 'list_perm_tag_1', 'TagValues': ['val2', 'val1']}, {'TagKey': 'list_perm_tag_2', 'TagValues': ['val2']}]}}, 'Permissions': ['ALTER', 'DESCRIBE'], 'PermissionsWithGrantOption': [], 'LastUpdated': datetime.datetime(2023, 10, 5, 20, 25, 33, 277000, tzinfo=tzlocal()), 'LastUpdatedBy': 'arn:aws:iam::123456789012:user/abcdefghi'}


### Grant permissions to the resouce link using my account (Account1) and grant permissions to admin role using Account2's expression

In [38]:
p_lf.grant_permissions(
    Principal={
        'DataLakePrincipalIdentifier': USER_ARN
    },
    Resource={
        'LFTagPolicy': {
            'CatalogId' : ACC2_ACCOUNT_ID,
            'ResourceType': 'DATABASE',
            'Expression' : [
                {
                    'TagKey': LF_TAG_1_NAME,
                    'TagValues': [
                        'val1',
                        'val2'
                    ]
                },
                {
                    'TagKey': LF_TAG_2_NAME,
                    'TagValues': [
                        'val2'
                    ]
                }
            ]
        }
    },
    Permissions=['DESCRIBE', 'ALTER']
)

p_lf.grant_permissions(
    Principal={
        'DataLakePrincipalIdentifier': USER_ARN
    },
    Resource={
        'LFTagPolicy': {
            'CatalogId' : ACC2_ACCOUNT_ID,
            'ResourceType': 'TABLE',
            'Expression': [
                {
                    'TagKey': LF_TAG_1_NAME,
                    'TagValues': [
                        'val1',
                        'val2'
                    ]
                },
                {
                    'TagKey': LF_TAG_2_NAME,
                    'TagValues': [
                        'val2'
                    ]
                }
            ]
        }
    },
    Permissions=['SELECT', 'DESCRIBE']
)


{'ResponseMetadata': {'RequestId': '361b1995-f553-439b-874c-ba6bba6177ac',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 06 Oct 2023 00:30:52 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '361b1995-f553-439b-874c-ba6bba6177ac',
   'cache-control': 'no-cache'},
  'RetryAttempts': 0}}

In [39]:
db = p_glue.get_database(
    Name=DB_RL_NAME
)

print(db['Database'])

{'Name': 'lftags_test_database', 'CreateTableDefaultPermissions': [], 'TargetDatabase': {'CatalogId': '987654321098', 'DatabaseName': 'lftags_test_database'}, 'CatalogId': '123456789012'}


### List Perms on the user
This shows the perms on the USER (the user is an LF-Admin, and we are providing the DataLakePrincipalIdentifer which is why its showing grantable)

In [44]:
perms = p_lf.list_permissions(
    Principal={
        'DataLakePrincipalIdentifier': USER_ARN
    },
    Resource={
        'LFTagPolicy': {
            'CatalogId' : ACC2_ACCOUNT_ID,
            'ResourceType': 'TABLE',
            'Expression': [
                {
                    'TagKey': LF_TAG_1_NAME,
                    'TagValues': [
                        'val1',
                        'val2'
                    ]
                },
                {
                    'TagKey': LF_TAG_2_NAME,
                    'TagValues': [
                        'val2'
                    ]
                }
            ]
        }
    }   
)

for perm in perms['PrincipalResourcePermissions']:
    print(perm)

{'Principal': {'DataLakePrincipalIdentifier': 'arn:aws:iam::123456789012:role/admin'}, 'Resource': {'LFTagPolicy': {'CatalogId': '987654321098', 'ResourceType': 'TABLE', 'Expression': [{'TagKey': 'list_perm_tag_1', 'TagValues': ['val2', 'val1']}, {'TagKey': 'list_perm_tag_2', 'TagValues': ['val2']}]}}, 'Permissions': ['DESCRIBE', 'INSERT', 'SELECT'], 'PermissionsWithGrantOption': ['DESCRIBE', 'INSERT', 'SELECT']}
